In [34]:
from comet_ml import Experiment
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchtext.legacy.data import Field, TabularDataset, BucketIterator
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import f1_score
from tqdm import tqdm_notebook as tqdm
from google.cloud import secretmanager
import numpy as np
import json
import pickle
import warnings
# Filter warnings because Pytorch depricating Field and other functions,
# but not clear on what to use instead yet
warnings.filterwarnings('ignore')

In [35]:
# !pip3 install -U pip setuptools wheel
# !pip3 install -U spacy
# !python -m spacy download en_core_web_trf

In [36]:
with open('../data/raw/comet_cred.json') as f:
    comet_creds = json.load(f)

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
source_folder = "../data/interim/"

In [38]:
hyper_params = {
    "tokenize": "spacy",
    "lower": True,
    "batch_size": 16,
    "hidden_size": 64,
    "embedding_size": 32,
    "num_classes": 3,
    "num_layers": 1,
    "learning_rate": 0.001,
    "min_freq": 5,
    "bidirectional": True,
    "num_epochs": 75,
    "disabled": False,
    "dropout_p": 0.6,
    "save_model_path": "../models/pytorch_model.pt",
    "save_vocab_path": "../models/pytorch_vocab.pkl"
}

In [39]:
experiment = Experiment(api_key=comet_creds["api_key"], project_name=comet_creds["project_name"],
                        workspace=comet_creds["workspace"], disabled=hyper_params["disabled"])
experiment.log_parameters(hyper_params)

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/jahnic/general/92f5a90cf21a4f91abecbfa0b7f52fff
COMET INFO:   Parameters:
COMET INFO:     batch_size      : 16
COMET INFO:     bidirectional   : True
COMET INFO:     disabled        : 1
COMET INFO:     dropout_p       : 0.6
COMET INFO:     embedding_size  : 32
COMET INFO:     hidden_size     : 64
COMET INFO:     learning_rate   : 0.001
COMET INFO:     lower           : True
COMET INFO:     min_freq        : 5
COMET INFO:     num_classes     : 3
COMET INFO:     num_epochs      : 75
COMET INFO:     num_layers      : 1
COMET INFO:     save_model_path : ../models/pytorch_model.pt
COMET INFO:     save_vocab_path : ../models/pytorch_vocab.pkl
COMET INFO:     tokenize        : spacy
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COME